## LangChain Expression Language(LCEL)


LCEL은 복잡한 LLM 체인과 워크플로우를 구성하기 위한 문법입니다.

이를 통해 복잡한 워크플로조차도 몇 줄의 코드로 간단하게 표현할 수 있습니다.


### LCEL 문법의 장점

- 사용 편의성과 유연성: 복잡한 워크플로를 쉽게 생성하고 확장할 수 있으며, 프로토타입에서 프로덕션으로의 전환이 매끄럽습니다.
- 성능 및 최적화: 병렬 실행과 지연 시간 감소로 성능을 향상시켜 실시간 애플리케이션에 유용합니다.
- 고급 기능: 재시도, 폴백, 스트리밍을 지원하여 신뢰성과 모니터링을 개선합니다.
- 커뮤니티와 채택: 긍정적인 평가를 받고 있으며, 커뮤니티 채택이 점점 증가하고 있습니다.


### LCEL 문법의 원리, [출처](https://github.com/langchain-ai/langchain/blob/dbb6b7b103d9c32cea46d3848839a4c9cbb493c3/libs/core/langchain_core/runnables/base.py#L564-L586)

```python
...
    def __or__(
        self,
        other: Union[
            Runnable[Any, Other],
            Callable[[Any], Other],
            Callable[[Iterator[Any]], Iterator[Other]],
            Mapping[str, Union[Runnable[Any, Other], Callable[[Any], Other], Any]],
        ],
    ) -> RunnableSerializable[Input, Other]:
        """Compose this Runnable with another object to create a RunnableSequence."""
        return RunnableSequence(self, coerce_to_runnable(other))

    def __ror__(
        self,
        other: Union[
            Runnable[Other, Any],
            Callable[[Other], Any],
            Callable[[Iterator[Other]], Iterator[Any]],
            Mapping[str, Union[Runnable[Other, Any], Callable[[Other], Any], Any]],
        ],
    ) -> RunnableSerializable[Other, Output]:
        """Compose this Runnable with another object to create a RunnableSequence."""
        return RunnableSequence(coerce_to_runnable(other), self)
...
```

Runnable 체인의 연결은 pipe operatore( | ) 또는 Runnable의 .pipe() 메서드를 사용합니다.

마찬가지로 | 기호를 사용하기 위해서 __or__ 와 __ror__ 을 구현하고 있습니다. 

In [1]:
# API KEY 값이 들어 있는 환경 변수 로드
# 새로운 실습 파일이 만들어 질 때마다 맨 상위에 항상 해당 코드를 넣어주세요.
from dotenv import load_dotenv

load_dotenv()

True

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다.

In [2]:
from langchain.prompts import PromptTemplate

# template 정의
template = "{topic} 주제에 대한 재미있는 영화 추천해줘."

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 주제에 대한 재미있는 영화 추천해줘.')

In [3]:
# prompt 생성
prompt = prompt_template.format(topic="로맨스")
prompt

'로맨스 주제에 대한 재미있는 영화 추천해줘.'

In [4]:
# prompt 생성
prompt = prompt_template.format(topic="액션")
prompt

'액션 주제에 대한 재미있는 영화 추천해줘.'

In [5]:
from langchain_core.output_parsers import StrOutputParser

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [6]:
from langchain_openai import ChatOpenAI

# 모델 정의
llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4o-mini",
)

### LCLE 방식으로 Chain 결합

![lcle.png](./images/lcle.png)

이제 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다.

```python
chain = prompt | model | output_parser
```


In [7]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 주제에 대한 재미있는 영화 추천해줘.")

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

chain = prompt | model | output_parser

### invoke() 메소드 호출

- python 딕셔너리 형태로 입력값을 전달합니다. (키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [8]:
# input 딕셔너리에 원하는 주제를 '액션'으로 설정합니다.
input = { "topic": "액션" }

In [9]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

'액션 주제의 재미있는 영화 몇 편을 추천해드릴게요!\n\n1. **존 윅 (John Wick)** - 키아누 리브스가 주연한 이 영화는 전직 킬러가 사랑하는 개를 잃고 복수를 다짐하는 이야기입니다. 화려한 액션과 스타일리시한 연출이 매력적입니다.\n\n2. **매드 맥스: 분노의 도로 (Mad Max: Fury Road)** - 포스트 아포칼립스 세계를 배경으로 한 이 영화는 스릴 넘치는 추격전과 강렬한 비주얼로 많은 사랑을 받았습니다.\n\n3. **다크 나이트 (The Dark Knight)** - 크리스토퍼 놀란 감독의 배트맨 시리즈 중 두 번째 작품으로, 조커 역의 히스 레저의 연기가 인상적입니다. 액션과 스릴이 가득한 영화입니다.\n\n4. **미션 임파서블: 폴아웃 (Mission: Impossible - Fallout)** - 톰 크루즈가 주연한 이 시리즈는 매번 새로운 액션과 스릴 넘치는 스토리로 관객을 사로잡습니다.\n\n5. **킹스맨: 시크릿 에이전트 (Kingsman: The Secret Service)** - 스파이 액션 영화로, 유머와 스타일이 잘 어우러진 작품입니다. 독특한 액션 장면이 인상적입니다.\n\n이 외에도 많은 훌륭한 액션 영화들이 있으니, 취향에 맞는 영화를 찾아보세요!'

## 병렬 처리

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel
from langchain.prompts import ChatPromptTemplate

# 1. OpenAI 모델 초기화
llm = ChatOpenAI(
    temperature=0.7,
    model_name="gpt-4o-mini",
)


chain1 = ChatPromptTemplate.from_template("{topic} 주제에 대한 재미있는 영화 추천해줘.") | llm
chain2 = (
    ChatPromptTemplate.from_template("{topic} 주제로 아재 개그 하나만 해줘.")
    | llm
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [11]:
%%time
chain1.invoke({"topic": "로맨스"},)

CPU times: user 6.25 ms, sys: 1.61 ms, total: 7.86 ms
Wall time: 6.54 s


AIMessage(content="로맨스 주제의 재미있는 영화 몇 편을 추천해드릴게요!\n\n1. **노트북 (The Notebook, 2004)** - 사랑의 힘과 인내를 다룬 이 영화는 한 남자가 그의 사랑을 위해 모든 것을 희생하는 이야기를 담고 있습니다. 감동적인 사랑 이야기가 많은 이들에게 감명을 주었습니다.\n\n2. **500일의 썸머 (500 Days of Summer, 2009)** - 비선형적인 스토리텔링과 독특한 캐릭터 설정으로 사랑의 시작과 끝을 다룬 이 영화는 현실적인 로맨스를 보여줍니다. 유머와 감동이 적절히 조화를 이룹니다.\n\n3. **러브 액츄얼리 (Love Actually, 2003)** - 여러 개의 사랑 이야기가 교차하는 이 영화는 크리스마스를 배경으로 하여 다양한 형태의 사랑을 다룹니다. 따뜻한 감성과 유머가 가득합니다.\n\n4. **내 여자친구의 결혼식 (My Best Friend's Wedding, 1997)** - 주인공이 자신의 베스트 프렌드의 결혼식에 참석하면서 벌어지는 사랑의 혼란을 그린 영화로, 유머와 로맨스가 잘 어우러져 있습니다.\n\n5. **비포 선라이즈 (Before Sunrise, 1995)** - 두 젊은이가 유럽 기차에서 우연히 만나 24시간 동안 함께 시간을 보내며 나누는 대화와 감정을 그린 영화로, 깊이 있는 로맨스를 느낄 수 있습니다.\n\n이 영화들은 각각 독특한 매력을 가지고 있어 로맨스를 좋아하는 분들에게 추천할 만합니다. 즐거운 감상 되세요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 378, 'prompt_tokens': 21, 'total_tokens': 399, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'r

In [12]:
%%time
chain2.invoke({"topic": "로맨스"},)

CPU times: user 6.83 ms, sys: 2.14 ms, total: 8.97 ms
Wall time: 1.13 s


AIMessage(content='왜 연애할 때는 항상 바람이 불까요? \n\n사랑이 바람을 타고 오니까요! 🌬️❤️', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 21, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-8233caf6-4a5b-4493-8785-797bbd2bf471-0', usage_metadata={'input_tokens': 21, 'output_tokens': 31, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
%%time
combined.invoke({"topic": "로맨스"},)

CPU times: user 17.1 ms, sys: 4.3 ms, total: 21.4 ms
Wall time: 5.3 s


{'joke': AIMessage(content="로맨스를 주제로 한 재미있는 영화 몇 편을 추천해드릴게요!\n\n1. **500일의 썸머 (500 Days of Summer)** - 사랑의 시작과 끝을 다룬 독특한 구조의 영화로, 현실적인 연애의 여러 단계를 유머러스하게 그려냅니다.\n\n2. **노트북 (The Notebook)** - 세대를 초월한 사랑 이야기를 담고 있으며, 감정적인 순간과 아름다운 배경이 인상적입니다.\n\n3. **러브 액츄얼리 (Love Actually)** - 여러 개의 사랑 이야기가 얽히며 펼쳐지는 크리스마스 시즌의 로맨틱 코미디로, 따뜻한 감동과 유머를 동시에 제공합니다.\n\n4. **이터널 선샤인 (Eternal Sunshine of the Spotless Mind)** - 기억을 지우는 과정을 통해 사랑의 복잡함을 탐구하는 독창적인 로맨스 영화입니다.\n\n5. **브리짓 존스의 일기 (Bridget Jones's Diary)** - 현실적인 연애의 어려움과 유머를 담고 있는 영화로, 주인공의 성장과 사랑을 그립니다.\n\n6. **그녀 (Her)** - 인공지능과의 사랑을 다룬 독특한 이야기로, 현대인의 고독과 사랑에 대한 깊은 성찰을 제공합니다.\n\n이 영화들은 각각 다른 매력을 가지고 있어 다양한 로맨스의 모습을 보여줍니다. 즐겁게 감상하세요!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 322, 'prompt_tokens': 21, 'total_tokens': 343, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0